In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import psycopg2
from config import db_password
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import explained_variance_score as evs
from sqlalchemy import create_engine

# connection function
def get_db_connection():
    conn = psycopg2.connect(database="N-Butane", user="postgres", password=db_password, host="127.0.0.1", port="5432")
    # will need to create a database, and make a config.py to hold the password 
    return conn

conn = get_db_connection()



# regression model function
def run_model(table_name):

    cursor = conn.cursor()

    # establish connect with postgres database to extract data
    cursor.execute('''SELECT * FROM %s''' % table_name)
    data = cursor.fetchall()
    df = pd.DataFrame(data)
    colnames = [desc[0] for desc in cursor.description]
    cursor.close()


    # set column names to the list pulled from SQL
    df.columns = colnames

    # Separate the Features (X) from the Target (y)
    y = df["n_butane"]
    X = df.drop(["n_butane", "Date"], axis = 1)

    # Split into train data and test data
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

    # Import Linear Regression Model
    model = LinearRegression()

    # Train the model
    model.fit(X_train, y_train)

    # Make Predictions
    y_pred = model.predict(X_test)
    results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)


    # Calculated Metrics
    # r2 score
    regression_score = r2_score(y_test, y_pred)

    # Mean Squared Error (MSE) and mean error
    from cmath import sqrt
    mean_sq_error = mse(y_test, y_pred)
    mean_error = sqrt(mean_sq_error).real

    # Mean Absolute Error (MAE)
    mean_abs_error = mae(y_test, y_pred)

    # explained variance score
    ev_score = evs(y_test, y_pred)

    # metrics dataframe
    scores = {
        "Data": [table_name],
        "R2_Score": [regression_score],
        "Mean_Error": [mean_error],
        "Mean_Absolute_Error": [mean_abs_error],
        "Estimated_Variance_Score": [ev_score]
}
    metrics = pd.DataFrame(scores)
    
    return results, metrics

In [2]:
proj_results, proj_metrics = run_model('project_data')

towers_results, towers_metrics = run_model('towers_data')

lab_rx_results, lab_rx_metrics = run_model('lab_rx')

lab_ta_results, lab_ta_metrics = run_model('lab_ta')

lab_tb_results, lab_tb_metrics = run_model('lab_tb')

lab_tc_results, lab_tc_metrics = run_model('lab_tc')

In [4]:
# making Results and Metrics tables in N-Butane Database
cursor = conn.cursor()
result_tables = ['proj_results',
                'towers_results',
                'lab_rx_results',
                'lab_ta_results',
                'lab_tb_results',
                'lab_tc_results']

metric_tables = ['proj_metrics', 
                'towers_metrics', 
                'lab_rx_metrics', 
                'lab_ta_metrics', 
                'lab_tb_metrics', 
                'lab_tc_metrics']

for i in result_tables:
    try:
        cursor.execute("CREATE TABLE %s (Data varchar, R2_Score real, Mean_Error real, Mean_Absolute_Error real, Estimated_Variance_Score real);" % i)
    except:
        print("I can't make the database!")

    conn.commit() 


for j in metric_tables:
    try:
        cursor.execute("CREATE TABLE %s (Predictions real, Actual real);" % j)
    except:
        print("I can't make the database!")

    conn.commit() 



In [5]:
# upload results and metrics to postgresql tables
connect = f"postgresql+psycopg2://postgres:" + db_password + "@127.0.0.1:5432/N-Butane" 
engine = create_engine(connect)

proj_results.to_sql('proj_results', con=engine, if_exists='replace', index=False)
proj_metrics.to_sql('proj_metrics', con=engine, if_exists='replace', index=False)

towers_results.to_sql('towers_results', con=engine, if_exists='replace', index=False)
towers_metrics.to_sql('towers_metrics', con=engine, if_exists='replace', index=False)

lab_rx_results.to_sql('lab_rx_results', con=engine, if_exists='replace', index=False)
lab_rx_metrics.to_sql('lab_rx_metrics', con=engine, if_exists='replace', index=False)

lab_ta_results.to_sql('lab_ta_results', con=engine, if_exists='replace', index=False)
lab_ta_metrics.to_sql('lab_ta_metrics', con=engine, if_exists='replace', index=False)

lab_tb_results.to_sql('lab_tb_results', con=engine, if_exists='replace', index=False)
lab_tb_metrics.to_sql('lab_tb_metrics', con=engine, if_exists='replace', index=False)

lab_tc_results.to_sql('lab_tc_results', con=engine, if_exists='replace', index=False)
lab_tc_metrics.to_sql('lab_tc_metrics', con=engine, if_exists='replace', index=False)

conn.autocommit=True